In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [15]:
import matplotlib.pyplot as plt

In [16]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'dataset/trainl',  # Update this path to your dataset location
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

Found 1066 images belonging to 3 classes.


In [18]:
validation_generator = train_datagen.flow_from_directory(
    'dataset/testl',  # Update this path to your dataset location
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 41 images belonging to 2 classes.


In [19]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [20]:
model = Model(inputs=base_model.input, outputs=predictions)

In [21]:
for layer in base_model.layers:
    layer.trainable = False

In [22]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/10
33/33 [==============================] - 18s 381ms/step - loss: -33.7901 - accuracy: 0.8385 - val_loss: 14.8667 - val_accuracy: 0.9688
Epoch 2/10
33/33 [==============================] - 13s 380ms/step - loss: -204.4347 - accuracy: 0.8511 - val_loss: 52.9917 - val_accuracy: 0.9688
Epoch 3/10
33/33 [==============================] - 13s 385ms/step - loss: -557.0446 - accuracy: 0.8501 - val_loss: 123.4340 - val_accuracy: 0.9688
Epoch 4/10
33/33 [==============================] - 13s 395ms/step - loss: -1158.9576 - accuracy: 0.8462 - val_loss: 232.4521 - val_accuracy: 0.9688
Epoch 5/10
33/33 [==============================] - 15s 446ms/step - loss: -1973.3984 - accuracy: 0.8472 - val_loss: 379.0146 - val_accuracy: 0.9688
Epoch 6/10
33/33 [==============================] - 12s 369ms/step - loss: -2910.7039 - accuracy: 0.8511 - val_loss: 557.3319 - val_accuracy: 0.9688
Epoch 7/10
33/33 [==============================] - 13s 395ms/step - loss: -4194.7837 - accuracy: 0.8501 - val_l

In [24]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy}")

2/2 [==============================] - 0s 95ms/step - loss: 1251.3167 - accuracy: 0.9756
Validation Accuracy: 0.9756097793579102


In [28]:
model.save('../saved_models/leaf.keras')
